This notebook demos how to make prompt functions -- that is, python functions that 
are defined by prompt templates.

# Pre-defined prompt functions

We have a few pre-defined prompt functions that come with the `oa` package. 
We'll start looking at those, and then see how to make them...

In [1]:
from oa import ask
from inspect import signature

print_signature = lambda func: print(signature(func))

list(ask.ai)[:10]

['prompt_template_improver',
 'user_story_to_code_for_dag',
 'prompt_template_starter',
 'make_synopsis',
 'description_to_software_specs',
 'define_jargon',
 'simple_tests_for_code',
 'suggest_names',
 'specs_to_code_for_dag']

## prompt_template_starter

Of particular interest to use here is an AI-enabled function that will help us make AI-enabled functions...

It has two inputs:
- a `task`, which is a specification of what we want a prompt template for
- `inputs`, which is a specification of what "variables" this prompt template should have, that is, what kinds of inputs we're going to ask the user

In [13]:
print_signature(ask.ai.prompt_template_starter)

(task, *, inputs=' ')


First we go through a bit of experimentation, trying out different ways of expressing what we want. 

In [14]:
template = ask.ai.prompt_template_starter(
    task="""
    Help me learn a new language
    """,
    inputs = ""
)
print(template)

You are a language teacher. Teach me how to say {this} in the new language.


In [20]:
template = ask.ai.prompt_template_starter(
    task="""
    Help me learn new vocabulary words of a language that I'm learning
    by giving me some sentences that I should translate into the language I am learning.
    """,
    inputs = "language, vocabulary_words, number_of_sentences"
)
print(template)

You are a language learning expert. Translate the following sentences into {language} using the vocabulary words: {vocabulary_words}. Translate {number_of_sentences} sentences.


Then we settle on some template, possibly editing it a bit, and make a prompt function out of it.

In [22]:
from oa import prompt_function

template = """
You are a language learning expert. Translate the following sentences into {language:Japanese} 
using the vocabulary words: {vocabulary_words}. 
Give me a list of {number_of_sentences:3} to translate followed with an ANSWERS section where 
you'll list the actual translations.
"""
practice_vocab = prompt_function(template)
print_signature(practice_vocab)

(vocabulary_words, *, language='Japanese', number_of_sentences='3')


Let's try this function out:

In [23]:
print(practice_vocab("tasty, sushi, hello, please, thank you"))

1. Hello, please give me some tasty sushi.
2. Thank you for the tasty sushi.
3. Hello, may I please have some more tasty sushi?

ANSWERS:
1. こんにちは、美味しい寿司をください。
2. 美味しい寿司をありがとうございます。
3. こんにちは、美味しい寿司をもう少しいただけますか。


Let's have a look at what the underlying template looks like.

In [24]:
print(ask.ai.prompt_template_starter.template_original)


```
You are an expert prompt engineer.
I will give you a task below and would like you to write a “prompt template” to perform this task.
This task will be parametrized by some inputs, which should appear in the prompt template as placeholders, marked by braces {like} {this} (here, both “like” and “this” are names of inputs that would parametrize the prompt template). 
I might give you an explicit list of inputs, which will be a comma or space separated list of strings (to be more precise, python identifier strings (so only alphanumerics and underscores).
If I do give you this list of inputs (names), you should use them (all) in the prompt template, using the {braces} to show where these should be injected when making a prompt (I have code for that). 
You should use them all.
If I don’t give you this list of inputs (that is, my list of inputs is empty), you should come up with your own. Remember; I’m going to use this template to make a form to get inputs from the user, so I need this

## prompt_template_improver

So now we have a template that works. How can we improve it?

Let's use another (prompt-based) function for that

In [2]:
template = """
You are a language learning expert. Translate the following sentences into {language:Japanese} 
using the vocabulary words: {vocabulary_words}. 
Give me a list of {number_of_sentences:3} to translate followed with an ANSWERS section where 
you'll list the actual translations.
"""

In [3]:
print_signature(ask.ai.prompt_template_improver)

(prompt_template, *, specific_goals=' ', number_of_alternatives='3', include_additional_explanations='false', prompt_engineering_tips='false')


In [5]:
alternative_template_1 = ask.ai.prompt_template_improver(template)
print(alternative_template_1)

**Prompt Alternatives:**
1. You are a language learning expert. Translate the following sentences into {language:Japanese} using the vocabulary words: {vocabulary_words}. Give me a list of {number_of_sentences:3} to translate followed with an ANSWERS section where you'll list the actual translations.
  
2. As a language learning expert, I need you to translate the following sentences into {language:Japanese} with the provided vocabulary words: {vocabulary_words}. Provide a list of {number_of_sentences:3} to translate, followed by an ANSWERS section where the actual translations will be listed.

3. Calling all language learning experts! Your task is to translate the following sentences into {language:Japanese} using the vocabulary words: {vocabulary_words}. I need a list of {number_of_sentences:3} translations, followed by an ANSWERS section detailing the actual translations.




In [6]:
alternative_template_2 = ask.ai.prompt_template_improver(
    template,
    specific_goals='To stay more focused on the vocabulary words, departing from them as little as possible', 
    number_of_alternatives=2, 
    include_additional_explanations=True, 
    prompt_engineering_tips=True
)
print(alternative_template_2)

Alternative 1:
You are a language learning expert. Translate the following sentences into {language:Japanese} 
using the vocabulary words: {vocabulary_words}. 
Give me a list of {number_of_sentences:3} to translate followed with an ANSWERS section where 
you'll list the actual translations.

Alternative 2:
As a language learning expert, translate the sentences below into {language:Japanese} 
with the given vocabulary words: {vocabulary_words}. 
Provide {number_of_sentences:3} translations and list them in the ANSWERS section.

ADDITIONAL EXPLANATIONS:
In these alternatives, the focus is on the vocabulary words provided, ensuring that the translations stick closely to them. By structuring the prompt in a specific format, it can help guide the expert towards using the vocabulary effectively in the translations.

PROMPT ENGINEERING TIPS:
1. Encourage the expert to utilize the vocabulary words creatively by providing a variety of sentence structures to translate.
2. Remind the expert to pa

## changing the chat model

Not good enough?

Perhaps the prompt is good enough, but not the ai?

Let's try to change the model we're using!

In [11]:
from oa.base import list_engine_ids

list_engine_ids()


['whisper-1',
 'davinci-002',
 'dall-e-2',
 'tts-1-hd-1106',
 'tts-1-hd',
 'gpt-3.5-turbo',
 'gpt-3.5-turbo-0125',
 'gpt-3.5-turbo-instruct-0914',
 'gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo-instruct',
 'gpt-3.5-turbo-0301',
 'gpt-3.5-turbo-0613',
 'tts-1',
 'dall-e-3',
 'gpt-3.5-turbo-1106',
 'babbage-002',
 'gpt-4-0125-preview',
 'gpt-4-turbo-preview',
 'tts-1-1106',
 'text-embedding-3-large',
 'gpt-4-turbo-2024-04-09',
 'gpt-4-vision-preview',
 'text-embedding-3-small',
 'gpt-4',
 'text-embedding-ada-002',
 'gpt-4-1106-vision-preview',
 'gpt-4-1106-preview',
 'gpt-4-0613',
 'gpt-4-turbo']

In [15]:
from oa import prompt_function, chat
from functools import partial

smarter_chat = partial(chat, model='gpt-4')
smarter_prompt_function = partial(prompt_function, prompt_func=smarter_chat)
# and now let's make a smarter prompt_template_improver
# Since prompt_template_improver was already made with the default chat model,
# we need to go back to the template, and make a new function with the new model
smarter_prompt_template_improver = smarter_prompt_function(
    ask.ai.prompt_template_improver.template_original
)

In [17]:
template = """
You are a language learning expert. Translate the following sentences into {language:Japanese} 
using the vocabulary words: {vocabulary_words}. 
Give me a list of {number_of_sentences:3} to translate followed with an ANSWERS section where 
you'll list the actual translations.
"""
alternative_template_3 = smarter_prompt_template_improver(
    template,
    specific_goals='To stay more focused on the vocabulary words, departing from them as little as possible', 
    number_of_alternatives=2, 
    include_additional_explanations=True, 
    prompt_engineering_tips=True
)
print(alternative_template_3)

Alternative 1:
For our language mastery exercise today, you are to translate the following phrases into {language:Japanese}. You will be utilizing these key phrases: {vocabulary_words}. Provide an index of {number_of_sentences:3} notions to translate and follow it with a SOLUTIONS segment where you catalog the accurate translations.

Alternative 2:
You are recognized for your proficiency in languages. The task here requires you to translate the subsequent sentences into {language:Japanese}, with a focus on these keywords: {vocabulary_words}. Try to construct and translate {number_of_sentences:3} original lines, followed by an ANSWERS segment displaying the correct translations.

ADDITIONAL EXPLANATIONS:

Alternative 1 slightly modifies the tone and language of the original template. In this alternative, the wording is a bit more formal. 'mastery exercise' is used instead of 'learning', which might help the AI take a more scholarly approach to the task. Also, by renaming the 'ANSWERS' t

In [1]:
from oa.util import OPENAI_API_KEY_ENV_NAME, DFLT_TEMPLATES_SOURCE_ENV_NAME


In [2]:
str.strip('asdfasdf ')

'asdfasdf'

In [1]:
from oa.util import config_getter

print(config_getter.sources[0].rootdir)

from oa import chat
chat

TypeError: can only concatenate str (not "tuple") to str

In [6]:
import oa
oa

<module 'oa' from '/Users/thorwhalen/Dropbox/py/proj/t/oa/oa/__init__.py'>

In [7]:
from oa.util import config_getter

config_getter('OPENAI_API_KEY_ENV_NAME', default='OPENAI_API_KEY')

In [2]:
OPENAI_API_KEY_ENV_NAME = 'OPENAI_API_KEY'
DFLT_TEMPLATES_SOURCE_ENV_NAME = 'OA_DFLT_TEMPLATES_SOURCE'

FuncBasedGettableContainer(getter=<functools._lru_cache_wrapper object at 0x10a0cdbc0>, val_is_valid=<function always_true at 0x10a090310>, config_not_found_exceptions=(<class 'Exception'>,))

In [4]:
s['test123']

In [23]:
s.getter('test123')

KeyError: TypeError('write() argument must be str, not None')

In [4]:
class NonIterable:
    def __init__(self):
        self.__iter__ = None

    def __getitem__(self, index):
        if index >= 0 and index < 10:
            return index
        else:
            raise IndexError('Index out of range')

obj = NonIterable()
print(isinstance(obj, Iterable))  # This will return False


False


In [7]:
class NonIterable:
    def __init__(self):
        self.__iter__ = None

    def __getitem__(self, index):
        if index >= 0 and index < 10:
            return index
        else:
            raise IndexError('Index out of range')

obj = NonIterable()
print(isinstance(obj, Iterable))  # This will return False


False


# Your own templates

I mean, really we've already gone through the motions now, perhaps unknowlingly.

We took a pre existing ai-based function as our point of departure, but have seen how it itself was 
made from a prompt template string. 

In [21]:
from base64 import b64encode, b64decode

b64decode("base64,aW1wb3J0IHJlcXVlc3RzCgojIEZvbmN0aW9uIHBvdXIgZWZmZWN0dWV")

Error: Invalid base64-encoded string: number of data characters (61) cannot be 1 more than a multiple of 4

In [ ]:
"data:text/x-python;name=update_all_prompt.py;base64,aW1wb3J0IHJlcXVlc3RzCgojIEZvbmN0aW9uIHBvdXIgZWZmZWN0dWV"

In [22]:
# decode this base64 string:

t = "python;name=update_all_prompt.py;base64,aW1wb3J0IHJlcXVlc3RzCgojIEZvbmN0aW9uIHBvdXIgZWZmZWN0dWV"

tt = t.split(";")

# get the contents of this file


['python',
 'name=update_all_prompt.py',
 'base64,aW1wb3J0IHJlcXVlc3RzCgojIEZvbmN0aW9uIHBvdXIgZWZmZWN0dWV']

In [25]:


def extract_string_from_data_url(data_url):
    """Extract a string from a data URL."""
    import base64

    # Split the URL at the first comma to separate the metadata from the base64 content
    metadata, encoded = data_url.split(',', 1)

    # Ensure the base64 string is a multiple of 4 in length by padding with '='
    padding = 4 - len(encoded) % 4
    if padding and padding != 4:
        encoded += '=' * padding

    # Decode the base64 string
    original_string = base64.b64decode(encoded).decode('utf-8')

    return original_string


data_url = "data:text/x-python;name=update_all_prompt.py;base64,aW1wb3J0IHJlcXVlc3RzCgojIEZvbmN0aW9uIHBvdXIgZWZmZWN0dWV"

try:
    original_content = extract_string_from_data_url(data_url)
    print(original_content)
except Exception as e:
    print(f"An error occurred: {e}")




import requests

# Fonction pour effectue


In [26]:
t = {
  "files": [],
  "file": "data:text/plain;name=thor_whalen_cv.txt;base64,Q1VSUklDVUxVTSBWSVRBRQpUaG9yIFdoYWxlbiBDVVJSRU5UIFBPU0lUSU9OOiBEaXJlY3RvciBvZiBNYWNoaW5lIExlYXJuaW5nCkRFR1JFRTogTWF0aGVtYXRpY3MgUGguRC4gJiBDb21wdXRlciBTY2llbmNlIE1hc3RlcnMuCkxpZmU6IEJvcm4gaW4gdGhlIFUuUy4sIHJhaXNlZCBpbiBGcmFuY2UgKG1vc3RseSksIEl0YWx5IGFuZCBHZXJtYW55LiBVUyBjaXRpemVuLgpXb3JrOiBDb25zdWx0YW50ICYgQW5hbHlzdDogRGF0YSBTY2llbmNlLCBNYXRoZW1hdGljcywgU3RhdGlzdGljYWwgTW9kZWxpbmcsIE9wdGltaXphdGlvbiwgQ29tcHV0ZXIgU2NpZW5jZSBOYXR1cmFsIExhbmd1YWdlczogRW5nbGlzaCwgRnJlbmNoIChiaWxpbmd1YWwpLCBJdGFsaWFuICh3b3JraW5nIGtub3dsZWRnZSksIEdlcm1hbiAoY29udmVyc2F0aW9uYWwpIENvbXB1dGluZyBMYW5ndWFnZXM6IFB5dGhvbiBFeHBlcnQuIEhpc3RvcmljYWw6IEMsIE1hdGxhYiwgUiwgSmF2YSwgVkJBLiBQcmVoaXN0b3JpY2FsOiBCYXNpYywgUGFzY2FsLCBMaXNwIEVtYWlsOiB0aG9yLmMud2hhbGVuQGdtYWlsLmNvbSBQaG9uZTogKzEtOTI5LTQyMi0xMTM0CldPUksKSSBoYXZlIHdvcmtlZCBwcmltYXJpbHkgYXMgYW4gaW5kZXBlbmRlbnQgbWFjaGluZSBsZWFybmluZyBjb25zdWx0YW50IGFuZCB0cmFpbmVyIHNpbmNlIHllYXIgMjAwMC4gTXkgd29yayByZXF1aXJlZCBhIGNyb3NzLWZ1bmN0aW9uYWwgYXBwcm9hY2ggdG8gcHJvYmxlbSBzb2x2aW5nIGluIGEgdmFyaWV0eSBvZiBzZWN0b3JzLiBJdCB1c3VhbGx5IHN0YXJ0ZWQgd2l0aCBmb3JtYWxpemluZyB3aGF0IHRoZSBDLWxldmVscyB3YW50ZWQsIGRlZmluaW5nIGFuZCBtZWFzdXJpbmcgb2JqZWN0aXZlcywgZGV2ZWxvcGluZyBtb2RlbHMsIGFuZCBmaW5hbGx5IHdvcmtpbmcgd2l0aCBleGlzdGluZyBkZXYgdGVhbXMgdG8gZ2V0IHRoZSBzb2x1dGlvbnMgaW50ZWdyYXRlZC4KSSBjby1mb3VuZGVkIE90b1NlbnNlLCBhIHNvdW5kIHJlY29nbml0aW9uIGNvbXBhbnkuIFdlIHNvbGQgaXQgdG8gQW5hbG9nIERldmljZXMgaW4gMjAxOCBhbmQgSSBoYXZlIGJlZW4gd29ya2luZyB3aXRoIHRoZW0gc2luY2UgdGhlbiwgZmlyc3QgYXMgYSBEaXJlY3RvciBvZiBNYWNoaW5lIExlYXJuaW5nLCBmb2N1c2VkIG9uIG9wZW4tc291cmNlIHRvb2xzIHRvIGZhY2lsaXRhdGUgY29sbGFib3JhdGlvbiBmb3IgdGhlIGRldmVsb3BtZW50IG9mIGVuZC10by1lbmQgZnJhbWV3b3JrcyBhbmQgcGxhdGZvcm1zIGZvciBzaWduYWwgTUwuCiAgQW5hbG9nIERldmljZXMKKFNlbWljb25kdWN0b3JzKQpPdG9TZW5zZQooU291bmQgUmVjb2duaXRpb24pClRhZ0NvbW1hbmRlcgooVGFnIE1hbmFnZW1lbnQgU3lzdGVtKQpFeHBlZGlhCihIb3RlbCBCb29raW5nKQoyMDE4LSBEaXJlY3RvciBvZiBNYWNoaW5lIExlYXJuaW5nIChBbmFsb2cgRGV2aWNlcyBib3VnaHQgT3RvU2Vuc2UpIHByZXNlbnQgLSBNYW5hZ2luZyBSJkQgaW4gSW9UIGRhdGEgbWFjaGluZSBsZWFybmluZyBhbmQgcGxhdGZvcm1zCi0gRGV2ZWxvcCB0b29scyB0byBhY2NlbGVyYXRlIGVuZC10by1lbmQgTUwgUE9DIGRldmVsb3BtZW50CjIwMTYtIENUTyBhbmQgY28tZm91bmRlciBvZiBzdGFydHVwCjIwMTggLSBEYXRhIFNjaWVuY2UgZm9yIHNvdW5kIHJlY29nbml0aW9uIEFJIHN5c3RlbXMKLSBNYW5hZ2VkIGJhY2tlbmQgYW5kIGZyb250ZW5kIHByb2plY3RzIGZvciBzb3VuZCByZWNvZ25pdGlvbiB0b29scwoyMDE0LSBTZW5pb3IgRGF0YSBTY2llbnRpc3QKMjAxNiAtIFNhbGVzIFRyYWplY3RvcnkgQW5hbHlzaXMgLSBNdWx0aS1jaGFubmVsIEF0dHJpYnV0aW9uCjIwMTEtIDIwMTQKLSBTZWFyY2ggRW5naW5lIE1hcmtldGluZyBPcHRpbWl6YXRpb24KLSBTdXBwbHkgYW5kIERlbWFuZCBncmFudWxhcml6YXRpb24sIGFsaWdubWVudCwgYW5kIHByaWNpbmcgLSBTZW1hbnRpYyBBbmFseXNpcyBhbmQgQ29uc3VtZXIgQmVoYXZpb3IKRnJvbSAyMDAxLTIwMTEgSSB3b3JrZWQgYXMgYW4gaW5kZXBlbmRlbnQgY29uc3VsdGFudC4gQmVsb3cgaXMgYSBsaXN0IG9mIGNvbXBhbmllcyBJIHdvcmtlZCBmb3IsIGFuZCBhIHNob3J0IHdvcmsgZGVzY3JpcHRpb24uIFRoZSB0aW1lIHBlcmlvZHMgb2YgdGhlIHByb2plY3Rz4oCUbmVpdGhlciBjb250aWd1b3VzIG5vciBzZXBhcmF0ZeKAlHdpbGwgYmUgb21pdHRlZCBmb3IgcmVhZGFiaWxpdHkuClNhbm9tYQooTWVkaWEpCkVhc3l2b3lhZ2UKKEZhcmUgY29tcGFyYXRvcikKRG90ZG90ZG90CihNb2JpbGUgUmVhZGVyIEFwcCkKRmlyc3QgQWZmaWxpYXRpb24KKEludGVybmV0IE1hcmtldGluZykKU2NlbnRyaWMKKERhdGEgTWFuYWdlbWVudCkKLSBFeGFtaW5lZCBvbmxpbmUgbWFya2V0aW5nIHByb2Nlc3MgYW5kIHN0cmF0ZWdpZXMsIGludGVncmF0aW9uIG9mIGRhdGEgc2NpZW5jZSB0ZWFtcyAtIFByZXNlbnRlZCBhbGdvcml0aG1zIHRvIGltcHJvdmUgc2VhcmNoIG1hcmtldGluZwotIEFpcmZhcmUgZm9yZWNhc3RpbmcgLSBEYXRhIEFuYWx5c2lzCi0gTmF0dXJhbCBsYW5ndWFnZSBwcm9jZXNzaW5nIC0gRG9jdW1lbnQgZmluZ2VycHJpbnRpbmcKLSBEZXNpZ25lZCBhbmQgZGV2ZWxvcGVkIGF1dG9tYXRpYyB0YXJnZXRpbmcgYW5kIGNhbXBhaWduIG9wdGltaXphdGlvbiBzeXN0ZW0gZm9yIGFmZmlsaWF0aW9uIGFuZCBkaXJlY3QgbWFpbCBtYXJrZXRpbmcKLSBUaW1lIHNlcmllcyBhbmFseXNpcyBhbmQgZm9yZWNhc3Rpbmcgb2YgdmFyaW91cyBtYXJrZXRpbmcgdmFyaWFibGVzCi0gU2VhcmNoIGVuZ2luZSBtYXJrZXRpbmc6IEZvcmVjYXN0aW5nLCByaXNrIG1hbmFnZW1lbnQgYW5kIGJpZCBvcHRpbWl6YXRpb24KLSBSZXNlYXJjaCBhbmQgZGV2ZWxvcG1lbnQgaW4gZGF0YSBtYW5hZ2VtZW50IGFuZCBpbmZvcm1hdGlvbiByZXRyaWV2YWwuIC0gRGV2ZWxvcGVkIGdlbmVyYWwgZHVwbGljYXRlIGRldGVjdGlvbiBzeXN0ZW0gLS0gd3JvdGUgYW5kIGZpbGVkIHBhdGVudC4KLSBBdXRvbWF0aWMgZG9jdW1lbnQgY2xhc3NpZmljYXRpb24KaVZpVml0eQooU3RvcmFnZSBNYW5hZ2VtZW50IGFuZCBOZXR3b3JraW5nKQpLTSBDb25zdWx0aW5nCihLbm93bGVkZ2UgTWFuYWdlbWVudCkKTWV0cm9uCihTY2llbnRpZmljIENvbnN1bHRpbmcpCklLT04gKG5vdyBSaWNvaCkgKE9mZmljZSBTb2x1dGlvbnMpCk51cnNpbmcgSG9tZSBRdWFsaXR5CihRSVMgZXhwZXJ0cykKLSBJbXByb3ZlZCBleGlzdGluZyBtZXRob2RzIGZvciBSQUlEIHN5c3RlbXMgYW5kIENSQyBjYWxjdWxhdGlvbnMKLSBXcm90ZSBwYXRlbnQgKFVTIHBhdGVudCAjNiw4MjMsNDI1KSBvbiBub3ZlbCBlcmFzdXJlIGNvZGVzIGZvciBSQUlEIHN5c3RlbXMgLSBBbmFseXplZCBoYXNoIGZ1bmN0aW9uc+KAmSBlZmZlY3RpdmVuZXNzIGFuZCAoY2lyY3VpdCBkZXNpZ24pIG9wdGltYWxpdHkKLSBDb25kdWN0ZWQgd29ya3Nob3BzIG9uIGRpdmVyc2UgbWF0aGVtYXRpY2FsIHRlY2huaXF1ZXMKLSBDb25kdWN0ZWQgd29ya3Nob3BzIG9uIGRhdGEgbWluaW5nLCBmb3JlY2FzdGluZywgYXJ0aWZpY2lhbCBpbnRlbGxpZ2VuY2UsIGV4cGVydGlzZSBzeXN0ZW1pemF0aW9uIGFuZCBrbm93bGVkZ2UgZXh0cmFjdGlvbi4KLSBXb3JrZWQgb24gZXhwcmVzc2luZyBrbm93bGVkZ2UgbWFuYWdlbWVudCBwcm9ibGVtcyBpbiBtYXRoZW1hdGljYWwgZm9ybQotIERldmVsb3BlZCB0aGVvcnkgYW5kIHNvZnR3YXJlIHRvb2wgZm9yIGRpc3RyaWJ1dGVkIGluZmVyZW5jZSBpbiBCYXllc2lhbiBuZXR3b3JrcyAtIFJlc2VhcmNoZWQgYW5kIHRlc3RlZCBNb250ZSBDYXJsbyBtZXRob2RzIGZvciBtdWx0aXBsZSB0YXJnZXQgdHJhY2tpbmcKLSBNb2RlbGluZyBpbmNlbnRpdmUgYW5hbHlzaXMgYW5kIGFsaWdubWVudAotIFdyb3RlIHNvZnR3YXJlIHRvb2wgZm9yIGNvbXBlbnNhdGlvbiBwbGFuIGFuYWx5c2lzCi0gRGV2ZWxvcGVkIGFuIGFwcHJvYWNoIHRvIGFzc2VzcyBwcmVjaXNpb24gYW5kIG9wdGltaXplIHJpc2sgYW5kIGNvc3QgZHVyaW5nIFFJUyAocXVhbGl0eSBpbmRpY2F0b3Igc2NvcmUpIGZhY2lsaXR5IHF1YWxpdHkgYXNzZXNzbWVudC4gRGV2ZWxvcGVkIHNpbXVsYXRpb25zLgpPdGhlciBtaXNjZWxsYW5lb3VzIHByb2plY3RzIGluY2x1ZGUKSWRlbnRpZmljYXRpb24gaW5mb3JtYXRpb24gb2YgbW91c2UgbW92ZW1lbnRzIGFuZCBrZXlzdHJva2UgcGF0dGVybnMgQmV0dGluZyAobnVtYmVyIGNvbWJpbmF0aW9uKSBjaG9pY2VzIG9mIG5hdGlvbmFsIGxvdHRlcnkKRW1lcmdlbmNlICYgc3ByZWFkIG9mIG5ldyB3b3JkcyBhbmQgZXhwcmVzc2lvbiBpbiB0aGUgYmxvZ29zcGhlcmUgQWRkaWN0aW9uIHBhdHRlcm5zIGFuZCBkeW5hbWljcwpERUdSRUVTIC8gRURVQ0FUSU9OCk9ubGluZSBwb2tlciBiZXR0aW5nIHBhdHRlcm5zIEF1Y3Rpb24gZHluYW1pY3MKU29jaWFsIG5ldHdvcmsgc3RydWN0dXJlIG9mIGJsb2dzIE11c2ljIGluZm9ybWF0aW9uIHJldHJpZXZhbAogUGguRC4gaW4gTWF0aGVtYXRpY3MKTS5TLiBpbiBDb21wdXRlciBTY2llbmNlCkxpY2VuY2UgZGUgTWF0aMOpbWF0aXF1ZXMgKE0uUy4gaW4gTWF0aCkgRC5FLlUuRy4gQSBNUE0gKEIuUy4gaW4gTWF0aC9QaHlzaWNzKSBHcmFkdWF0ZSBTdHVkaWVzCkNsYXNzaWNhbCBhbmQgSmF6eiBtdXNpY2FsIHN0dWRpZXMKRW1vcnkgVW5pdmVyc2l0eSAyMDAzIEVtb3J5IFVuaXZlcnNpdHkgMjAwMyBVbml2ZXJzaXTDqSBkZSBOaWNlIFNvcGhpYSBBbnRpcG9saXMsIE5pY2UsIEZyYW5jZSAxOTk2IFVuaXZlcnNpdMOpIGRlIE5pY2UgU29waGlhIEFudGlwb2xpcywgTmljZSwgRnJhbmNlIDE5OTQgVGVjaG5pc2NoZSBVbml2ZXJzaXTDpHQgQmVybGluLCBHZXJtYW55IDE5OTctMTk5OCBDb25zZXJ2YXRvaXJlIE5hdGlvbmFsIGRlIFLDqWdpb24gZGUgTmljZSwgRnJhbmNlIDE5ODUtMTk5NQpURUFDSElORyBFWFBFUklFTkNFClRhdWdodCBpbiBzZXZlcmFsIFVuaXZlcnNpdGllcyAobmFtZWx5LCBJbGxpbm9pcyBTdGF0ZSBVbml2ZXJzaXR5LCBFbW9yeSBVbml2ZXJzaXR5LCBKb2hucyBIb3BraW5zIENlbnRlciBmb3IgVGFsZW50ZWQgWW91dGgpIGFjcm9zcyB0aGUgVS5TLkEgJiB2YXJpb3VzIHdvcmtzaG9wcyBpbiB2YXJpb3VzIHN1YmplY3RzIHJlbGV2YW50IHRvIHRoZSBjbGllbnQuIFN1YmplY3RzIGluY2x1ZGU6CiBNYXJrZXRpbmcgbWF0aGVtYXRpY3MgUHJvYmFiaWxpdHkgYW5kIFN0YXRpc3RpY3MgVGhlb3J5IG9mIENvbXB1dGluZwpNYXRoZW1hdGljcyBmb3IgQ29tcHV0ZXIgU2NpZW5jZSBBZHZhbmNlZCBkYXRhIGFuYWx5c2lzCkRhdGEgU2NpZW5jZSBpbiBQeXRob24KUEFURU5UUyAmIEFDQURFTUlDIEFSVElDTEVTCkV4cGxvaXRpbmcgdW5jZXJ0YWludHkKT3B0aW1pemluZyBpbiBjb21wbGV4IHNldHRpbmdzIERhdGEgcmVwcmVzZW50YXRpb24gYW5kIGluZGV4YXRpb24KIEJlbG93IGlzIGEgbm9uLWV4aGF1c3RpdmUgbGlzdCBvZiBwdWJsaWNhdGlvbnMuCjEuIFN5bnRhY3RpYyBzeXN0ZW0gZm9yIHNvdW5kIHJlY29nbml0aW9uLCB3LyBDaHJpc3RpYW4sIFVTIFBhdGVudCAjVVMyMDE4MDI2ODg0NEExCjIuIFNvdW5kLXJlY29nbml0aW9uIHN5c3RlbSBiYXNlZCBvbiBhIHNvdW5kIGxhbmd1YWdlIGFuZCBhc3NvY2lhdGVkIGFubm90YXRpb25zLCB3LyBDaHJpc3RpYW4sIFVTIFBhdGVudCAjVVMyMDE4MDI1NDA1NEExCjMuIFN5c3RlbSBhbmQgbWV0aG9kIGZvciBpbXBsZW1lbnRpbmcgYWR2YW5jZWQgUkFJRCB1c2luZyBhIHNldCBvZiB1bmlxdWUgbWF0cmljZXMgYXMgQ29lZmZpY2llbnRzLCB3LyBHb3NoLCBKYWluLCBVUyBQYXRlbnQgIzYsODIzLDQyNQooYXNzaWduZWU6IGlWaXZpdHksIEluYy4pCjQuIEludGVsbGlnZW50IEdlbmVyYWwgRHVwbGljYXRlIE1hbmFnZW1lbnQgU3lzdGVtIHcvIEt1cmFuZGUuIFVTIFBhdGVudCAyMDA3LzAwNTA0MjMgKGFzc2lnbmVlOiBTY2VudHJpYywgSW5jKQo1LiBNZWFzdXJpbmcgRGlzY29udGludWl0eSBpbiBCaW5hcnkgTG9uZ2l0dWRpbmFsIERhdGEsIHcvIE1pcmlhbSBCb2VyaTogU29jaW9sb2dpY2FsIG1ldGhvZHMgJiByZXNlYXJjaC4gNDMoMikgKDIwMTQpCjYuIE9uIEgtaW1tZXJzaW9ucywgdy8gRmVycmFyYSwgVGFuc2V5LCBHb3VsZDogR3JhcGggVGhlb3J5IDU3KDMpICgyMDA4KQo3LiBTdWJkaXZpc2lvbiBFeHRlbmRpYmlsaXR5LCB3LyBHb3VsZDogR3JhcGhzIENvbWJpbi4gMjMoMikgKDIwMDcpCjguIERpc3RhbmNlIGJldHdlZW4gdHdvIGstc2V0cyBhbmQgcGF0aC1zeXN0ZW0gZXh0ZW5kaWJpbGl0eSwgdy8gR291bGQ6IEFycyBDb21iaW5hdG9yaWEgNzkgKDIwMDYKOS4gT24gSC1saW5rZWQgR3JhcGhzOiBVbmlmeWluZyBjb25uZWN0aXZpdHksIHcvIEZlcnJhcmEsIFRhbnNleSwgR291bGQ6IEdyYXBocyBDb21iaW4uIDIyKDIpICgyMDA2KQoxMC4gRWRnZS1kaXNqb2ludCBIYW1pbHRvbmlhbiBjeWNsZXMgaW4gYmlwYXJ0aXRlIGdyYXBocywgdy8gRmVycmFyYSwgVGFuc2V5LCBHb3VsZCwgRGlzY3JldGUgTWF0aGVtYXRpY3MgMzA5KDEyKSAoMjAwOSkKMTEuIFBhbi1rLWxpbmthZ2UgaW4gZGVuc2UgZ3JhcGhzLCB3LyBHb3VsZCwgUG93ZWxsLCBXYWduZXI6IERpc2NyZXRlIE1hdGhlbWF0aWNzIDMwOSgxMCkgKDIwMDkpCjEyLiBJcnJlZ3VsYXJpdHkgU3RyZW5ndGggb2YgRGlncmFwaHMsIHcvIEZlcnJhcmEsIEdpbGJlcnQsIEphY29ic29uOiBEaXNjcmV0ZSBNYXRoZW1hdGljcyAzMDkoMTkpICgyMDA5KQ=="
}

extract_string_from_data_url(t["file"])


'CURRICULUM VITAE\nThor Whalen CURRENT POSITION: Director of Machine Learning\nDEGREE: Mathematics Ph.D. & Computer Science Masters.\nLife: Born in the U.S., raised in France (mostly), Italy and Germany. US citizen.\nWork: Consultant & Analyst: Data Science, Mathematics, Statistical Modeling, Optimization, Computer Science Natural Languages: English, French (bilingual), Italian (working knowledge), German (conversational) Computing Languages: Python Expert. Historical: C, Matlab, R, Java, VBA. Prehistorical: Basic, Pascal, Lisp Email: thor.c.whalen@gmail.com Phone: +1-929-422-1134\nWORK\nI have worked primarily as an independent machine learning consultant and trainer since year 2000. My work required a cross-functional approach to problem solving in a variety of sectors. It usually started with formalizing what the C-levels wanted, defining and measuring objectives, developing models, and finally working with existing dev teams to get the solutions integrated.\nI co-founded OtoSense, 

In [ ]:
  "file": "data:text/plain;name=thor_whalen_cv.txt;base64,Q1VSUklDVUxVTSBWSVRBRQpUaG9yIFdoYWxlbiBDVVJSRU5UIFBPU0lUSU9OOiBEaXJlY3RvciBvZiBNYWNoaW5lIExlYXJuaW5nCkRFR1JFRTogTWF0aGVtYXRpY3MgUGguRC4gJiBDb21wdXRlciBTY2llbmNlIE1hc3RlcnMuCkxpZmU6IEJvcm4gaW4gdGhlIFUuUy4sIHJhaXNlZCBpbiBGcmFuY2UgKG1vc3RseSksIEl0YWx5IGFuZCBHZXJtYW55LiBVUyBjaXRpemVuLgpXb3JrOiBDb25zdWx0YW50ICYgQW5hbHlzdDogRGF0YSBTY2llbmNlLCBNYXRoZW1hdGljcywgU3RhdGlzdGljYWwgTW9kZWxpbmcsIE9wdGltaXphdGlvbiwgQ29tcHV0ZXIgU2NpZW5jZSBOYXR1cmFsIExhbmd1YWdlczogRW5nbGlzaCwgRnJlbmNoIChiaWxpbmd1YWwpLCBJdGFsaWFuICh3b3JraW5nIGtub3dsZWRnZSksIEdlcm1hbiAoY29udmVyc2F0aW9uYWwpIENvbXB1dGluZyBMYW5ndWFnZXM6IFB5dGhvbiBFeHBlcnQuIEhpc3RvcmljYWw6IEMsIE1hdGxhYiwgUiwgSmF2YSwgVkJBLiBQcmVoaXN0b3JpY2FsOiBCYXNpYywgUGFzY2FsLCBMaXNwIEVtYWlsOiB0aG9yLmMud2hhbGVuQGdtYWlsLmNvbSBQaG9uZTogKzEtOTI5LTQyMi0xMTM0CldPUksKSSBoYXZlIHdvcmtlZCBwcmltYXJpbHkgYXMgYW4gaW5kZXBlbmRlbnQgbWFjaGluZSBsZWFybmluZyBjb25zdWx0YW50IGFuZCB0cmFpbmVyIHNpbmNlIHllYXIgMjAwMC4gTXkgd29yayByZXF1aXJlZCBhIGNyb3NzLWZ1bmN0aW9uYWwgYXBwcm9hY2ggdG8gcHJvYmxlbSBzb2x2aW5nIGluIGEgdmFyaWV0eSBvZiBzZWN0b3JzLiBJdCB1c3VhbGx5IHN0YXJ0ZWQgd2l0aCBmb3JtYWxpemluZyB3aGF0IHRoZSBDLWxldmVscyB3YW50ZWQsIGRlZmluaW5nIGFuZCBtZWFzdXJpbmcgb2JqZWN0aXZlcywgZGV2ZWxvcGluZyBtb2RlbHMsIGFuZCBmaW5hbGx5IHdvcmtpbmcgd2l0aCBleGlzdGluZyBkZXYgdGVhbXMgdG8gZ2V0IHRoZSBzb2x1dGlvbnMgaW50ZWdyYXRlZC4KSSBjby1mb3VuZGVkIE90b1NlbnNlLCBhIHNvdW5kIHJlY29nbml0aW9uIGNvbXBhbnkuIFdlIHNvbGQgaXQgdG8gQW5hbG9nIERldmljZXMgaW4gMjAxOCBhbmQgSSBoYXZlIGJlZW4gd29ya2luZyB3aXRoIHRoZW0gc2luY2UgdGhlbiwgZmlyc3QgYXMgYSBEaXJlY3RvciBvZiBNYWNoaW5lIExlYXJuaW5nLCBmb2N1c2VkIG9uIG9wZW4tc291cmNlIHRvb2xzIHRvIGZhY2lsaXRhdGUgY29sbGFib3JhdGlvbiBmb3IgdGhlIGRldmVsb3BtZW50IG9mIGVuZC10by1lbmQgZnJhbWV3b3JrcyBhbmQgcGxhdGZvcm1zIGZvciBzaWduYWwgTUwuCiAgQW5hbG9nIERldmljZXMKKFNlbWljb25kdWN0b3JzKQpPdG9TZW5zZQooU291bmQgUmVjb2duaXRpb24pClRhZ0NvbW1hbmRlcgooVGFnIE1hbmFnZW1lbnQgU3lzdGVtKQpFeHBlZGlhCihIb3RlbCBCb29raW5nKQoyMDE4LSBEaXJlY3RvciBvZiBNYWNoaW5lIExlYXJuaW5nIChBbmFsb2cgRGV2aWNlcyBib3VnaHQgT3RvU2Vuc2UpIHByZXNlbnQgLSBNYW5hZ2luZyBSJkQgaW4gSW9UIGRhdGEgbWFjaGluZSBsZWFybmluZyBhbmQgcGxhdGZvcm1zCi0gRGV2ZWxvcCB0b29scyB0byBhY2NlbGVyYXRlIGVuZC10by1lbmQgTUwgUE9DIGRldmVsb3BtZW50CjIwMTYtIENUTyBhbmQgY28tZm91bmRlciBvZiBzdGFydHVwCjIwMTggLSBEYXRhIFNjaWVuY2UgZm9yIHNvdW5kIHJlY29nbml0aW9uIEFJIHN5c3RlbXMKLSBNYW5hZ2VkIGJhY2tlbmQgYW5kIGZyb250ZW5kIHByb2plY3RzIGZvciBzb3VuZCByZWNvZ25pdGlvbiB0b29scwoyMDE0LSBTZW5pb3IgRGF0YSBTY2llbnRpc3QKMjAxNiAtIFNhbGVzIFRyYWplY3RvcnkgQW5hbHlzaXMgLSBNdWx0aS1jaGFubmVsIEF0dHJpYnV0aW9uCjIwMTEtIDIwMTQKLSBTZWFyY2ggRW5naW5lIE1hcmtldGluZyBPcHRpbWl6YXRpb24KLSBTdXBwbHkgYW5kIERlbWFuZCBncmFudWxhcml6YXRpb24sIGFsaWdubWVudCwgYW5kIHByaWNpbmcgLSBTZW1hbnRpYyBBbmFseXNpcyBhbmQgQ29uc3VtZXIgQmVoYXZpb3IKRnJvbSAyMDAxLTIwMTEgSSB3b3JrZWQgYXMgYW4gaW5kZXBlbmRlbnQgY29uc3VsdGFudC4gQmVsb3cgaXMgYSBsaXN0IG9mIGNvbXBhbmllcyBJIHdvcmtlZCBmb3IsIGFuZCBhIHNob3J0IHdvcmsgZGVzY3JpcHRpb24uIFRoZSB0aW1lIHBlcmlvZHMgb2YgdGhlIHByb2plY3Rz4oCUbmVpdGhlciBjb250aWd1b3VzIG5vciBzZXBhcmF0ZeKAlHdpbGwgYmUgb21pdHRlZCBmb3IgcmVhZGFiaWxpdHkuClNhbm9tYQooTWVkaWEpCkVhc3l2b3lhZ2UKKEZhcmUgY29tcGFyYXRvcikKRG90ZG90ZG90CihNb2JpbGUgUmVhZGVyIEFwcCkKRmlyc3QgQWZmaWxpYXRpb24KKEludGVybmV0IE1hcmtldGluZykKU2NlbnRyaWMKKERhdGEgTWFuYWdlbWVudCkKLSBFeGFtaW5lZCBvbmxpbmUgbWFya2V0aW5nIHByb2Nlc3MgYW5kIHN0cmF0ZWdpZXMsIGludGVncmF0aW9uIG9mIGRhdGEgc2NpZW5jZSB0ZWFtcyAtIFByZXNlbnRlZCBhbGdvcml0aG1zIHRvIGltcHJvdmUgc2VhcmNoIG1hcmtldGluZwotIEFpcmZhcmUgZm9yZWNhc3RpbmcgLSBEYXRhIEFuYWx5c2lzCi0gTmF0dXJhbCBsYW5ndWFnZSBwcm9jZXNzaW5nIC0gRG9jdW1lbnQgZmluZ2VycHJpbnRpbmcKLSBEZXNpZ25lZCBhbmQgZGV2ZWxvcGVkIGF1dG9tYXRpYyB0YXJnZXRpbmcgYW5kIGNhbXBhaWduIG9wdGltaXphdGlvbiBzeXN0ZW0gZm9yIGFmZmlsaWF0aW9uIGFuZCBkaXJlY3QgbWFpbCBtYXJrZXRpbmcKLSBUaW1lIHNlcmllcyBhbmFseXNpcyBhbmQgZm9yZWNhc3Rpbmcgb2YgdmFyaW91cyBtYXJrZXRpbmcgdmFyaWFibGVzCi0gU2VhcmNoIGVuZ2luZSBtYXJrZXRpbmc6IEZvcmVjYXN0aW5nLCByaXNrIG1hbmFnZW1lbnQgYW5kIGJpZCBvcHRpbWl6YXRpb24KLSBSZXNlYXJjaCBhbmQgZGV2ZWxvcG1lbnQgaW4gZGF0YSBtYW5hZ2VtZW50IGFuZCBpbmZvcm1hdGlvbiByZXRyaWV2YWwuIC0gRGV2ZWxvcGVkIGdlbmVyYWwgZHVwbGljYXRlIGRldGVjdGlvbiBzeXN0ZW0gLS0gd3JvdGUgYW5kIGZpbGVkIHBhdGVudC4KLSBBdXRvbWF0aWMgZG9jdW1lbnQgY2xhc3NpZmljYXRpb24KaVZpVml0eQooU3RvcmFnZSBNYW5hZ2VtZW50IGFuZCBOZXR3b3JraW5nKQpLTSBDb25zdWx0aW5nCihLbm93bGVkZ2UgTWFuYWdlbWVudCkKTWV0cm9uCihTY2llbnRpZmljIENvbnN1bHRpbmcpCklLT04gKG5vdyBSaWNvaCkgKE9mZmljZSBTb2x1dGlvbnMpCk51cnNpbmcgSG9tZSBRdWFsaXR5CihRSVMgZXhwZXJ0cykKLSBJbXByb3ZlZCBleGlzdGluZyBtZXRob2RzIGZvciBSQUlEIHN5c3RlbXMgYW5kIENSQyBjYWxjdWxhdGlvbnMKLSBXcm90ZSBwYXRlbnQgKFVTIHBhdGVudCAjNiw4MjMsNDI1KSBvbiBub3ZlbCBlcmFzdXJlIGNvZGVzIGZvciBSQUlEIHN5c3RlbXMgLSBBbmFseXplZCBoYXNoIGZ1bmN0aW9uc+KAmSBlZmZlY3RpdmVuZXNzIGFuZCAoY2lyY3VpdCBkZXNpZ24pIG9wdGltYWxpdHkKLSBDb25kdWN0ZWQgd29ya3Nob3BzIG9uIGRpdmVyc2UgbWF0aGVtYXRpY2FsIHRlY2huaXF1ZXMKLSBDb25kdWN0ZWQgd29ya3Nob3BzIG9uIGRhdGEgbWluaW5nLCBmb3JlY2FzdGluZywgYXJ0aWZpY2lhbCBpbnRlbGxpZ2VuY2UsIGV4cGVydGlzZSBzeXN0ZW1pemF0aW9uIGFuZCBrbm93bGVkZ2UgZXh0cmFjdGlvbi4KLSBXb3JrZWQgb24gZXhwcmVzc2luZyBrbm93bGVkZ2UgbWFuYWdlbWVudCBwcm9ibGVtcyBpbiBtYXRoZW1hdGljYWwgZm9ybQotIERldmVsb3BlZCB0aGVvcnkgYW5kIHNvZnR3YXJlIHRvb2wgZm9yIGRpc3RyaWJ1dGVkIGluZmVyZW5jZSBpbiBCYXllc2lhbiBuZXR3b3JrcyAtIFJlc2VhcmNoZWQgYW5kIHRlc3RlZCBNb250ZSBDYXJsbyBtZXRob2RzIGZvciBtdWx0aXBsZSB0YXJnZXQgdHJhY2tpbmcKLSBNb2RlbGluZyBpbmNlbnRpdmUgYW5hbHlzaXMgYW5kIGFsaWdubWVudAotIFdyb3RlIHNvZnR3YXJlIHRvb2wgZm9yIGNvbXBlbnNhdGlvbiBwbGFuIGFuYWx5c2lzCi0gRGV2ZWxvcGVkIGFuIGFwcHJvYWNoIHRvIGFzc2VzcyBwcmVjaXNpb24gYW5kIG9wdGltaXplIHJpc2sgYW5kIGNvc3QgZHVyaW5nIFFJUyAocXVhbGl0eSBpbmRpY2F0b3Igc2NvcmUpIGZhY2lsaXR5IHF1YWxpdHkgYXNzZXNzbWVudC4gRGV2ZWxvcGVkIHNpbXVsYXRpb25zLgpPdGhlciBtaXNjZWxsYW5lb3VzIHByb2plY3RzIGluY2x1ZGUKSWRlbnRpZmljYXRpb24gaW5mb3JtYXRpb24gb2YgbW91c2UgbW92ZW1lbnRzIGFuZCBrZXlzdHJva2UgcGF0dGVybnMgQmV0dGluZyAobnVtYmVyIGNvbWJpbmF0aW9uKSBjaG9pY2VzIG9mIG5hdGlvbmFsIGxvdHRlcnkKRW1lcmdlbmNlICYgc3ByZWFkIG9mIG5ldyB3b3JkcyBhbmQgZXhwcmVzc2lvbiBpbiB0aGUgYmxvZ29zcGhlcmUgQWRkaWN0aW9uIHBhdHRlcm5zIGFuZCBkeW5hbWljcwpERUdSRUVTIC8gRURVQ0FUSU9OCk9ubGluZSBwb2tlciBiZXR0aW5nIHBhdHRlcm5zIEF1Y3Rpb24gZHluYW1pY3MKU29jaWFsIG5ldHdvcmsgc3RydWN0dXJlIG9mIGJsb2dzIE11c2ljIGluZm9ybWF0aW9uIHJldHJpZXZhbAogUGguRC4gaW4gTWF0aGVtYXRpY3MKTS5TLiBpbiBDb21wdXRlciBTY2llbmNlCkxpY2VuY2UgZGUgTWF0aMOpbWF0aXF1ZXMgKE0uUy4gaW4gTWF0aCkgRC5FLlUuRy4gQSBNUE0gKEIuUy4gaW4gTWF0aC9QaHlzaWNzKSBHcmFkdWF0ZSBTdHVkaWVzCkNsYXNzaWNhbCBhbmQgSmF6eiBtdXNpY2FsIHN0dWRpZXMKRW1vcnkgVW5pdmVyc2l0eSAyMDAzIEVtb3J5IFVuaXZlcnNpdHkgMjAwMyBVbml2ZXJzaXTDqSBkZSBOaWNlIFNvcGhpYSBBbnRpcG9saXMsIE5pY2UsIEZyYW5jZSAxOTk2IFVuaXZlcnNpdMOpIGRlIE5pY2UgU29waGlhIEFudGlwb2xpcywgTmljZSwgRnJhbmNlIDE5OTQgVGVjaG5pc2NoZSBVbml2ZXJzaXTDpHQgQmVybGluLCBHZXJtYW55IDE5OTctMTk5OCBDb25zZXJ2YXRvaXJlIE5hdGlvbmFsIGRlIFLDqWdpb24gZGUgTmljZSwgRnJhbmNlIDE5ODUtMTk5NQpURUFDSElORyBFWFBFUklFTkNFClRhdWdodCBpbiBzZXZlcmFsIFVuaXZlcnNpdGllcyAobmFtZWx5LCBJbGxpbm9pcyBTdGF0ZSBVbml2ZXJzaXR5LCBFbW9yeSBVbml2ZXJzaXR5LCBKb2hucyBIb3BraW5zIENlbnRlciBmb3IgVGFsZW50ZWQgWW91dGgpIGFjcm9zcyB0aGUgVS5TLkEgJiB2YXJpb3VzIHdvcmtzaG9wcyBpbiB2YXJpb3VzIHN1YmplY3RzIHJlbGV2YW50IHRvIHRoZSBjbGllbnQuIFN1YmplY3RzIGluY2x1ZGU6CiBNYXJrZXRpbmcgbWF0aGVtYXRpY3MgUHJvYmFiaWxpdHkgYW5kIFN0YXRpc3RpY3MgVGhlb3J5IG9mIENvbXB1dGluZwpNYXRoZW1hdGljcyBmb3IgQ29tcHV0ZXIgU2NpZW5jZSBBZHZhbmNlZCBkYXRhIGFuYWx5c2lzCkRhdGEgU2NpZW5jZSBpbiBQeXRob24KUEFURU5UUyAmIEFDQURFTUlDIEFSVElDTEVTCkV4cGxvaXRpbmcgdW5jZXJ0YWludHkKT3B0aW1pemluZyBpbiBjb21wbGV4IHNldHRpbmdzIERhdGEgcmVwcmVzZW50YXRpb24gYW5kIGluZGV4YXRpb24KIEJlbG93IGlzIGEgbm9uLWV4aGF1c3RpdmUgbGlzdCBvZiBwdWJsaWNhdGlvbnMuCjEuIFN5bnRhY3RpYyBzeXN0ZW0gZm9yIHNvdW5kIHJlY29nbml0aW9uLCB3LyBDaHJpc3RpYW4sIFVTIFBhdGVudCAjVVMyMDE4MDI2ODg0NEExCjIuIFNvdW5kLXJlY29nbml0aW9uIHN5c3RlbSBiYXNlZCBvbiBhIHNvdW5kIGxhbmd1YWdlIGFuZCBhc3NvY2lhdGVkIGFubm90YXRpb25zLCB3LyBDaHJpc3RpYW4sIFVTIFBhdGVudCAjVVMyMDE4MDI1NDA1NEExCjMuIFN5c3RlbSBhbmQgbWV0aG9kIGZvciBpbXBsZW1lbnRpbmcgYWR2YW5jZWQgUkFJRCB1c2luZyBhIHNldCBvZiB1bmlxdWUgbWF0cmljZXMgYXMgQ29lZmZpY2llbnRzLCB3LyBHb3NoLCBKYWluLCBVUyBQYXRlbnQgIzYsODIzLDQyNQooYXNzaWduZWU6IGlWaXZpdHksIEluYy4pCjQuIEludGVsbGlnZW50IEdlbmVyYWwgRHVwbGljYXRlIE1hbmFnZW1lbnQgU3lzdGVtIHcvIEt1cmFuZGUuIFVTIFBhdGVudCAyMDA3LzAwNTA0MjMgKGFzc2lnbmVlOiBTY2VudHJpYywgSW5jKQo1LiBNZWFzdXJpbmcgRGlzY29udGludWl0eSBpbiBCaW5hcnkgTG9uZ2l0dWRpbmFsIERhdGEsIHcvIE1pcmlhbSBCb2VyaTogU29jaW9sb2dpY2FsIG1ldGhvZHMgJiByZXNlYXJjaC4gNDMoMikgKDIwMTQpCjYuIE9uIEgtaW1tZXJzaW9ucywgdy8gRmVycmFyYSwgVGFuc2V5LCBHb3VsZDogR3JhcGggVGhlb3J5IDU3KDMpICgyMDA4KQo3LiBTdWJkaXZpc2lvbiBFeHRlbmRpYmlsaXR5LCB3LyBHb3VsZDogR3JhcGhzIENvbWJpbi4gMjMoMikgKDIwMDcpCjguIERpc3RhbmNlIGJldHdlZW4gdHdvIGstc2V0cyBhbmQgcGF0aC1zeXN0ZW0gZXh0ZW5kaWJpbGl0eSwgdy8gR291bGQ6IEFycyBDb21iaW5hdG9yaWEgNzkgKDIwMDYKOS4gT24gSC1saW5rZWQgR3JhcGhzOiBVbmlmeWluZyBjb25uZWN0aXZpdHksIHcvIEZlcnJhcmEsIFRhbnNleSwgR291bGQ6IEdyYXBocyBDb21iaW4uIDIyKDIpICgyMDA2KQoxMC4gRWRnZS1kaXNqb2ludCBIYW1pbHRvbmlhbiBjeWNsZXMgaW4gYmlwYXJ0aXRlIGdyYXBocywgdy8gRmVycmFyYSwgVGFuc2V5LCBHb3VsZCwgRGlzY3JldGUgTWF0aGVtYXRpY3MgMzA5KDEyKSAoMjAwOSkKMTEuIFBhbi1rLWxpbmthZ2UgaW4gZGVuc2UgZ3JhcGhzLCB3LyBHb3VsZCwgUG93ZWxsLCBXYWduZXI6IERpc2NyZXRlIE1hdGhlbWF0aWNzIDMwOSgxMCkgKDIwMDkpCjEyLiBJcnJlZ3VsYXJpdHkgU3RyZW5ndGggb2YgRGlncmFwaHMsIHcvIEZlcnJhcmEsIEdpbGJlcnQsIEphY29ic29uOiBEaXNjcmV0ZSBNYXRoZW1hdGljcyAzMDkoMTkpICgyMDA5KQ=="


##